This notebook estimates the model $y = \exp(X \beta) + u$ where $u \sim N(0, 1.5)$ with indirect inference, using a linear regression as the auxiliary model.

Load necessary packages.

In [1]:
using Random
using CSV
using DataFrames
using Distributions
using LinearAlgebra
using Statistics
using Optim

Sets random seed, for generating random numbers to be used in the simulations.

In [2]:
Random.seed!(243587)

TaskLocalRNG()

Loads the data and stores in arrays X and y.

In [3]:
data = CSV.read("data_ii.csv", DataFrame)

const X = Matrix(data[!, [:x1, :x2]])
const y = Vector(data[!, :y]);

Draws random numbers for the simulations.

In [4]:
const M = 10
const u = rand(Normal(0, sqrt(1.5)), length(y), M);

Defines the estimator for the linear regression.

In [5]:
function ols(x, y; add_constant=true)
    if add_constant == true
        n = size(x, 1)
        x = hcat(ones(n), x)
    end
    x \ y # this is the same as (x'*x)^{-1} * x'*y, but faster and more accurate
end

ols (generic function with 1 method)

Estimates auxiliary model with the observed data and stores parameters in $\widehat{\beta}$.

In [6]:
const β̂ = ols(X, y)

3-element Vector{Float64}:
 2.831630852490789
 0.7352811810457907
 1.2756697273637128

Defines the structural model, to be used for simulating variable y.

In [7]:
function G(X, u, θ)
    exp.(hcat(ones(size(X, 1)), X) * θ) + u
end

G (generic function with 1 method)

Defines the Wald criterion function.

In [8]:
wald(β̃; β̂=β̂) = sum((β̂ - β̃).^2)

wald (generic function with 1 method)

Defines the objective function to be optimized. The trick here is to define an objective function that, for each value of $\theta$ simulates endogenous variables, estimates the auxiliary model, and returns the Wald criterion.

In [9]:
function obj(θ; X=X, u=u, β̂=β̂)
    ỹ = [G(X, u[:, m], θ) for m in 1:M] # this simulates the data
    β̃ = mean([ols(X, ỹ[m]) for m in 1:M]) # this estimates aux model on simulated data, and then averages parameters
    return wald(β̃; β̂=β̂)
end

obj (generic function with 1 method)

Optimizes the criterion function.

In [10]:
θ0 = ols(X, y) # initial vector of parameters
opt = optimize(θ -> obj(θ; X=X, u=u, β̂=β̂), θ0, NelderMead())

 * Status: success

 * Candidate solution
    Final objective value:     2.509667e-09

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    87
    f(x) calls:    164


The estimated parameters $\theta$ are

In [11]:
opt.minimizer

3-element Vector{Float64}:
 0.9161777416152237
 0.2539215090768394
 0.4384842282660998

Compare with the true values.